
# Getting Started

The [Maintaining Personalized Experiences with Machine Learning](https://aws.amazon.com/solutions/implementations/maintaining-personalized-experiences-with-ml/) solution helps you to create custom recommendation at scale. This solution streamlines and accelerates the development and deployment of your personalization workloads throug end-to-end automation and scheduling of updates for resources within the Amazon Personalize service.

The solution is launched as a CloudFormation template, and deploys the following infrastructure:

![Architecture Diagram](../../static/imgs/personalize-solution-architecture.jpg)

1. An Amazon Simple Storage Service (Amazon S3) bucket used to store personalization data and configuration files.
1. An AWS Lambda function initiated when new or updated personalization configuration is uploaded to the personalization data bucket.
1. An AWS Step Functions workflow to manage all of the resources of an Amazon Personalize dataset group (including datasets, schemas, event tracker, filters, solutions, campaigns, and batch inference jobs).
1. Amazon CloudWatch metrics for Amazon Personalize for each new trained solution version are added to help you evaluate the performance of a model over time.
1. An Amazon Simple Notification Service (SNS) topic and subscription to notify an administrator when the maintenance workflow has completed via email.
1. Amazon DynamoDB tracks the scheduled events configured for Amazon Personalize to fully or partially retrain Amazon Personalize solutions, import or reimport datasets, and perform batch inference jobs.
1. An AWS Step Functions workflow tracks the current running scheduled events, and invoke step functions to perform Amazon Personalize solution maintenance (creating new solution versions, updating campaigns), import updated datasets, and perform batch inference.
1. A set of maintenance AWS step functions to create new dataset import jobs on schedule; perform Amazon Personalize solution FULL retraining on schedule (and update associated campaigns); perform Amazon Personalize solution UPDATE retraining on schedule (and update associated campaigns); and create batch inference jobs.
1. An Amazon EventBridge event bus, where resource status notification updates are posted throughout the AWS Step functions workflow
1. A command line interface (CLI) lets existing resources be imported and allows schedules to be established for resources that already exist in Amazon Personalize

# Deploying the Solution

Run the next cell to generate a "Launch Stack" link for the region your notebook is running in. For the purposes of this lab, please do not change the stack name (but feel free to provide an Email address in the prompt for parameters, to enable notifications.

**Note**: if notifications are desired, you must provide an email address **and** accept the SNS subscription confirmation via email.

In [ ]:
# Run this cell to generate a "Launch Stack" link for your region (or use the generated, us-east-1 link below)
import boto3
import solution_helper

# Adjust this to the name you would like your stack to have
solution_helper.STACK_NAME = "PersonalizeStack"

# Display the Quick Launch Link
solution_helper.show_quick_launch_link()


While you wait for the stack to deploy (it takes about five minutes), check out some of the Solution Details below, then proceed to the next step.

# Solution Details

- Visit the [Solution Landing Page](https://aws.amazon.com/solutions/implementations/maintaining-personalized-experiences-with-ml/) to learn more about the solution
- Check out the [Solution Implementation Guide](https://docs.aws.amazon.com/solutions/latest/maintaining-personalized-experiences-with-ml/welcome.html) to learn how to use and operate the solution
- View the [Solution Source Code](https://github.com/aws-solutions/maintaining-personalized-experiences-with-machine-learning) if you need to customize and extend the solution for your own use case

## Uploading data

Let's get the bucket that the CloudFormation stack deployed. We will be uploading our data and configuration file(s) to trigger the automation. 

In [ ]:
# Check the status of the CloudFormation Stack
# Deployment takes about five minutes. 
from importlib import reload 
reload(solution_helper)

solution_helper.wait_for_stack()
bucket_name = solution_helper.get_stack_bucket()
table_name = solution_helper.get_stack_dynamo()



Now that we have the bucket name, we can copy over our data. There are three ways to upload your datasets to S3:

1. Using the AWS Console
2. Using the AWS CLI 
3. Using the boto3 SDK (we do this next) 

Using the drop-down, select your domain and click "Deploy" to have the SDK upload the sample datasets of your choice. Note that the datasets will be uploaded, but until the configuration file is created, they will not be imported into Amazon Personalize.


In [ ]:
# Run this cell to display a dropdown for each domain, and select `Deploy` to deploy the sample datasets for the desired domain
solution_helper.show_dataset_uploader(bucket_name=bucket_name)

## Starting the State Machine Execution

In order to have the solution automate resource creation in Amazon Personalize, we need to provide it with a parameters file that will tell our state machine which names and configurations we want in our Amazon Personalize deployment. 

The solution requires this file to be valid JSON and the file name must end in `.json`.

Using the drop-down, select your domain, and click "Generate" to have the notebook display your configuration. Once you have reviewed the configuration, click "Upload" to start creating resources in Amazon Personalize. Once again, there are three ways to upload your configuration to S3:

1. Using the AWS Console
2. Using the AWS CLI 
3. Using the boto3 SDK (we do this next)

In [ ]:
# Run this cell to display a dropdown for each domain, and select `Show` then `Deploy` to deploy the sample datasets for the desired domain
from importlib import reload; reload(solution_helper)
solution_helper.show_config_uploader(bucket_name=bucket_name)

## Validating your MLOps pipeline

The fastest way to check on the status of your pipeline is to go to the CloudWatch dashboard. In the AWS Console, you can navigate to the [Amazon CloudWatch console](https://console.aws.amazon.com/cloudwatch/home?#dashboards:). Here, a dashboard will be deployed with the name `PersonalizeSolution-PersonalizeSolution-<region>` where `<region>` is the region in which your dashboard is deployed. This dashboard tracks:

1. The number of configuration files processed (tracking success and failure)
2. The number of workflow jobs run (tracking success and failure)
3. Metrics around scheduler job creation
4. Metrics for Amazon Personalize resource creation

The solution also ensures that as solution versions are created in Amazon Personalize, Amazon CloudWatch metrics for each offline metric is created. This allows you to track the performance of your model in a directional sense. Depending on the stack name and solution trained, these metrics will be availalble as:

- `personalize_solution_<stack_name>` > `SolutionMetrics` > `<solutionArn>` > `<metric_name>`

The metrics documented [here](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) are published to CloudWatch by the solution as new Solution Versions are trained.

### Checking the status of workflow(s) in the AWS StepFunctions console:

The solution deploys four step functions into your account

1. `Personalize Workflow` - The main MLOps workflow
2. `Personalize Dataset Import` - The workflow used to generate new dataset imports on schedule
3. `Personalize Solution Maintenance` - The workflow used to create new solution versions (FULL or UPDATE retraining) and batch inference jobs on schedule
4. `Personalize Scheduler` - The workflow used to to schedule dataset imports and solution maintenance 


In [ ]:
# Run this cell to get the ARNs / console links of the step functions state machines listed above
from importlib import reload
reload(solution_helper)
stepfunctions = solution_helper.show_stack_stepfunctions()

### Additional information

Model training time will vary, and will take at least 30 minutes to complete. 

You can check the status of any of the state machines above in the console by:

1. Clicking **any execution** in the main MLOps workflow (1) step function to view its status

2. You can see which steps are currently running, highlited in blue

The solutions' step function definitions are tuned to automatically retry each step by querying the describe service APIs with a backoff rate, and accommodate retryable service quotas [documented here](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)

The main personalize workflow will take about 30 minutes to finish executing, which includes importing the datasets, training at least one solution, and deploying a campaign. **Note:** we are only training a SIMS model due to time constrains.

# Scheduling

You can set up scheduling for dataset import, solution version retraining, and batch inference jobs. To do so, upload a configuration file with scheduling enabled or use the `aws-solutions-scheduler` included in the solution to enable scheduling on an existing resource.

## Listing Schedules 

A list of schedules can be obtained by using the `list` command:

In [ ]:
# Run this code to list schedules: 
region = solution_helper.REGION
stack = solution_helper.STACK_NAME

!aws-solutions-scheduler --stack {stack} --region {region} list 

## Describe a Schedule

A schedule can be described by using the `describe` command and specifying the scheduled `task` name. Depending on the demo data and configuration deployed, you should use one of the following task names: 

- **CPG**: `personalize-dataset-import-immersion_day_cpg`
- **Media**: `personalize-dataset-import-immersion_day_media`
- **Retail**: `personalize-dataset-import-immersion_day_retail`

In [ ]:
# Change this task name to one of the task names listed above!
task_name = "personalize-dataset-import-immersion_day_retail"

!aws-solutions-scheduler --stack {stack} --region {region} describe --task {task_name}

## Update a Schedule

A schedule can be updated (or imported) using the `import-dataset-group` command. The solution supports cron-style expressions, as documented [here](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-create-rule-schedule.html#eb-cron-expressions).


In [ ]:
# Change the dataset group to be to immersion_day_media, immersion_day_cpg, or immersion_day_retail
dataset_group = "immersion_day_retail"
domain = dataset_group.split("_")[-1]

!aws-solutions-scheduler --stack {stack} --region {region} import-dataset-group --dataset-group {dataset_group} --path train/immersionday/{domain}/config.json --import-schedule "cron(0 0 ? * 2 *)" --full-schedule "immersion_{domain}_sims@cron(0 0 ? * 1 *)"

After running the update command, re-run the `list` and `describe` commands to view the updated list of tasks and display them.

## Deactivate or Activate a Schedule

Deactivating a schedule can be useful for maintenance operations.

### Deactivation 

Activated schedules can be deactivated

In [ ]:
!aws-solutions-scheduler --stack {stack} --region {region} deactivate --task solution-maintenance-full-immersion_{domain}_sims >/dev/null && aws-solutions-scheduler --stack {stack} --region {region} describe --task solution-maintenance-full-immersion_{domain}_sims

### Activation

Deactivated schedules can be activated

In [ ]:
!aws-solutions-scheduler --stack {stack} --region {region} activate --task solution-maintenance-full-immersion_{domain}_sims

# Cleanup

The stack can be cleaned up following the instructions [here](https://docs.aws.amazon.com/solutions/latest/maintaining-personalized-experiences-with-ml/uninstall-the-solution.html). This will remove the stack and schedules, but leave all resources untouched in Amazon Personalize. This process is automated for you by running the cells below.

## Cleanup Resources in Amazon Personalize:

Run the next cell to delete all the immersion day resources in Amazon Personalize created by this notebook

In [ ]:
solution_helper.delete_personalize_resources()

## Cleanup Schedules

Stop all scheduled tasks managed by the solution.

In [ ]:
print("Deactivating Tasks:")
!aws-solutions-scheduler --stack {stack} --region {region} list | jq -c ".tasks[]|@text" | xargs -L1 aws-solutions-scheduler --stack {stack} --region {region} deactivate --task 
print("Task Status:")
!aws-solutions-scheduler --stack {stack} --region {region} list | jq -c ".tasks[]|@text" | xargs -L1 aws-solutions-scheduler --stack {stack} --region {region} describe --task 

## Cleanup Stack Resources

You can now delete the stack:

In [ ]:
print(f"Deleting Stack: {solution_helper.STACK_NAME}")
solution_helper.STACK_NAME = "PersonalizeStack"
!aws cloudformation delete-stack --stack-name {solution_helper.STACK_NAME}
!aws cloudformation wait stack-delete-complete --stack-name {solution_helper.STACK_NAME}

Once you are sure the stack has been deleted, run the next cell to delete the bucket and dynamodb table created by the stack (by default, the solution will leave these resources in your account so as to not delete data unexpectedly).

In [ ]:
print(f"Deleting Bucket: {bucket_name}")
!aws s3 rb s3://{bucket_name} --force

print(f"Deleting Table: {table_name}")
!aws dynamodb delete-table --table-name {table_name}

All the solution resources have been cleaned from your account